# Ejercicio: análisis semántico de grupos de conversación

<img src="img/news.jpg">

En este ejercicio vamos a realizar el análisis semántico de unos grupos de conversación sobre noticias. Para ello emplearemos técnicas como Latent Dirichlet Allocation que nos permitan detectar temáticas de conversación de forma automática. Comprobaremos también si la detección de estas temáticas puede sernos de utilidad para un problema de clasificación supervisada. ¡Adelante!

## Instrucciones

A lo largo de este cuaderno encontrarás celdas vacías que tendrás que rellenar con tu propio código. Sigue las instrucciones del cuaderno y presta especial atención a los siguientes iconos:

<table>
<tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">Deberás responder a la pregunta indicada con el código o contestación que escribas en la celda inferior.</td></tr>
 <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">Esto es una pista u observación que te puede ayudar a resolver la práctica.</td></tr>
 <tr><td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">Este es un ejercicio avanzado y voluntario que puedes realizar si quieres profundar más sobre el tema. Te animamos a intentarlo para aprender más ¡Ánimo!</td></tr>
</table>

Para evitar problemas de compatibilidad y de paquetes no instalados, se recomienda ejecutar este notebook bajo uno de los [entornos recomendados de Text Mining](https://github.com/albarji/teaching-environments/tree/master/textmining).

Adicionalmente si necesitas consultar la ayuda de cualquier función python puedes colocar el cursor de escritura sobre el nombre de la misma y pulsar Mayúsculas+Shift para que aparezca un recuadro con sus detalles. Ten en cuenta que esto únicamente funciona en las celdas de código.

¡Adelante!

## Carga de datos

En primer lugar vamos a cargar el corpus con el que trabajaremos. Se trata del corpus **newsgroups20**, de referencia en el campo, y que está fácilmente disponible a través de scikit-learn. Con las siguientes instrucciones cargamos en memoria los datos del conjunto de entrenamiento y test del corpus, así como realizamos una limpieza básica:

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

Los textos vienen etiquetados según el grupo de conversación al que pertenencen. Como podemos ver, existen 20 grupos de conversación, y de ahí el nombre del corpus:

In [ ]:
set(newsgroups_train.target)

El significado de estos grupos de conversación numerados del 0 al 19 es el siguiente. Como vemos, aunque varios de los temas se centran en informática, existen otros grupos variados: automovilismo, deportes, religión, atletismo, ...

In [ ]:
newsgroups_train.target_names

Contamos además con bastantes textos en el conjunto de entrenamiento:

In [ ]:
len(newsgroups_train.data)

Veamos un ejemplo de un texto de un corpus, y el grupo al que está asociado:

In [ ]:
print("Texto:\n\n", newsgroups_train.data[0])
print("")
print("Grupo asociado:", newsgroups_train.target_names[newsgroups_train.target[0]])

## Análisis de temas con LDA

Ahora vamos a emplear **Latent Dirichlet Allocation** (LDA) para intentar descubrir los temas de conversación existentes. LDA es un método no supervisado, lo que significa que solo requiere de los textos para trabajar, y no necesita de información etiquetada sobre la temática de estos textos. En este corpus concreto ya conocemos de antemano los 20 temas en los que se agrupan los textos, lo cual es un campo de pruebas ideal para ver si LDA es capaz de encontrar a ciegas estos temas, o algunos similares.

### Limpieza de datos

Para obtener un análisis limpio necesitaremos tokenizar los textos, y eliminar símbolos de puntuación y aquellas palabras que no sean relevantes. Todo esto se puede hacer con los vectorizadores de scikit-learn que ya conocemos. En esta ocasión vamos a emplear TF-IDF, que al dar poco peso a palabras habituales del lenguaje es un buen punto de partida para detectar temas de conversación diferenciados. Comenzamos construyendo el vectorizador

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', min_df=10)
vectorizer.fit(newsgroups_train.data)

Ahora podemos aplicarlo sobre los datos de entrenamiento:

In [ ]:
vects = vectorizer.transform(newsgroups_train.data)

Con esto hemos obtenido una representación vectorizal de los textos, que como es habitual está almacenada en una matriz sparse para ahorrar espacio en memoria:

In [ ]:
vects

### Construcción del modelo LDA

La propia librería de aprendizaje automático scikit-learn incluye métodos muy prácticos para aplicar LDA. En particular, LDA está implementado en la clase **LatentDirichletAllocation**. Vamos a crear un transformador de esta clase, indicándole que queremos que trate de encontrar 20 temas en los datos.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=20)

Los objetos del tipo LatentDirichletAllocation siguen la misma interfaz que otros transformadores de scikit-learn como puede ser TfidfVectorizer. Esto es, disponen de un método **fit** para entrenarlos, y una vez entrenados pueden ser empleados para transformar otros conjuntos de datos.

Como hemos hecho arriba con TfidfVectorizer, vamos a entrenar nuestro modelo LDA. Solo debemos tener en cuenta que LDA no trabaja sobre los textos en bruto, sino que debe recibir una representación vectorial de los mismos. Aprovechando que ya la hemos calculado arriba, hacemos:

In [ ]:
lda.fit(vects)

Una vez entrenado el modelo podemos inspeccionarlo de varias formas. Probablemente el dato más interesante es analizar los componentes del modelo, que se refieren a la pertenencia de cada palabra del corpus a cada tema encontrado:

In [ ]:
lda.components_

In [ ]:
lda.components_.shape

¿Cómo interpretar esta matriz? La matriz tiene tantas filas como temas hayamos pedido a LDA que encuentre, y tantas columnas como palabras en el corpus. Esto significa que si cogemos una columna de esta matriz podremos saber cuán relacionada está con cada tema encontrado. Por ejemplo, la siguiente función toma una palabra, busca con qué índice ha sido codificado por TF-IDF, y nos devuelve los pesos para cada tema que ha encontrado LDA para ella:

In [ ]:
def findwordrelevances(vectorizer, lda, word):
    if word not in vectorizer.get_feature_names():
        print("La palabra '%s' no existe en el corpus, o se ha descartado por el vectorizador" % word)
        return
    idx = vectorizer.get_feature_names().index(word)
    print("La palabra '%s' tiene la siguiente relevancia por temas:" % word)
    print(lda.components_[:,idx])

In [ ]:
findwordrelevances(vectorizer, lda, "windows")

In [ ]:
findwordrelevances(vectorizer, lda, "god")

In [ ]:
findwordrelevances(vectorizer, lda, "jesus")

In [ ]:
findwordrelevances(vectorizer, lda, "car")

<table>
<tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">Prueba a buscar otras palabras que creas que puedan ser muy indicativas de uno de los temas que existen realmente en el corpus. ¿Ves en las relevancia de LDA que esté claramente posicionada a favor de uno de los temas?</td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table>
<tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">Ten en cuenta que LDA es un método estocástico, lo que significa que los temas encontrados pueden variar en cada ejecución. Si observas que las relevancias de palabras por temas que aparecen arriba no tienen sentido, prueba a reejecutar la construcción del modelo LDA.</td></tr>
</table>

También podemos hacer el análisis en otro sentido: ver qué palabras son las más asociadas con cada tema descubierto por LDA. Para ello vamos a basarnos en las siguientes funciones de utilidad:

In [ ]:
def print_top_words(vectorizer, lda, n_top_words=20):
    """Dado un vectorizador y en modelo LDA aplicado sobre él, imprime las palabras más relevantes de cada tema"""
    for topic_idx, topic in enumerate(lda.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join(top_words_topic(vectorizer, lda, topic_idx, n_top_words))
        print(message)
    print()
    
def top_words_topic(vectorizer, lda, topic_idx, n_top_words=20):
    """Devuelve una lista de las palabras más representativas para el i-ésimo tema de un modelo LDA"""
    feature_names = vectorizer.get_feature_names()
    topic = lda.components_[topic_idx]
    return [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]

In [ ]:
print_top_words(vectorizer, lda)

Podemos ayudarnos también de una visualización en forma de **nube de palabras** para entender mejor los temas generados. La siguiente función genera esta visualización.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud

def plot_topics(vectorizer, lda):
    """Genera una representación de nubes de palabras para los temas encontrados por LDA"""
    feature_names = vectorizer.get_feature_names()
    ntopics = lda.components_.shape[0]
    nrows = int(np.ceil(np.sqrt(ntopics)))
    ncols = int(np.ceil(ntopics / float(nrows)))
    fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(20,20))
    plainaxes = axs.ravel()
    for topic_idx, topic in enumerate(lda.components_):
        currentax = plainaxes[topic_idx]
        topidx = topic.argsort()
        plt.sca(currentax)
        wcloud = WordCloud(width=400, height=400, background_color='white')
        wcloud.generate_from_frequencies({feature_names[idx]: topic[idx] for idx in topidx})
        plt.imshow(wcloud, interpolation='bilinear')
        sns.despine()
        currentax.set_title("Topic %d" % topic_idx, fontsize=20)
        currentax.axis('off')
    plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9, wspace=0, hspace=0.1)

In [ ]:
plot_topics(vectorizer, lda)

Finalmente, podemos aplicar el modelo LDA entrenado para obtener la probabilidad de pertenencia a cada tema de un documento vectorizado cualquiera, aplicando la función transform. Por ejemplo, vamos a tomar el siguiente documento de entrenamiento:

In [ ]:
docindex = 2
print("Documento original:\n")
print(newsgroups_train.data[docindex])

Ahora le aplicamos la vectorización y el modelo LDA, y así obtenemos las siguientes pertenencias a cada tema:

In [ ]:
vectsample = vectorizer.transform([newsgroups_train.data[docindex]])
topics = lda.transform(vectsample)
print("\nProbabilidad de cada tema:\n")
print(topics)

¿Tiene sentido esta correspondencia temas? Podemos comprobarlo viendo qué palabras son las más representativas del tema que mayor probabilidad haya obtenido:

In [ ]:
print("\nTema más probable: %d\n" % np.argmax(topics))
print("Palabras más relevantes del tema:\n")
print(top_words_topic(vectorizer, lda, np.argmax(topics)))

Este proceso puede realizarse con cualquier texto, no solo con los textos que se han usado durante el entrenamiento. Por ejemplo:

In [ ]:
sampletext = """
We know that all mankind are fallen, we all bear the stain of Adams sin.
But Jesus came to redeem a people for Himself with His blood. Ephesians 1:7

Believers in Jesus, those who have accepted His Salvation, are now people from every tribe, race, nation and language. Revelation 5:9-10 But God originally chose a people group to be the ones who would be His Witnesses and to display His Light to the world. Those people failed in their task and were ejected for the holy Land in two dispersions.
Those of the second diaspora; the Jewish people have come back to a part of the holy Land, but still in unbelief and apostasy.

The Bible tells us in many prophesies; that all of Israel, who are by now an uncountable multitude, will return to the holy Land and will fulfil at last, God's plan for them.
The New Testament tells about a symbolic Olive Tree, one that has had all its branches removed. That Tree is Jesus and all who believe in Him, will be grafted into that Tree.
So whether we Christians are actual descendants of Jacob or not, the people who will fulfil God's Plan for a righteous people in His Land; will all be Christians. 
"""

vectsample = vectorizer.transform([sampletext])
topics = lda.transform(vectsample)
print("\nProbabilidad de cada tema:\n")
print(topics)
print("\nTema más probable: %d\n" % np.argmax(topics))
print("Palabras más relevantes del tema:\n")
print(top_words_topic(vectorizer, lda, np.argmax(topics)))

<table>
<tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">¿Tiene sentido el tema que se ha asignado al texto?
</table>

## LDA para problemas supervisados

Latent Dirichlet Analysis también puede emplearse para generar variables explicativas de utilidad (los topics) que puedan usarse para reforzar un sistema de clasificación supervisada. Vamos a ver a continuación cómo hacer esto.

En primer lugar vamos a construir un sistema de clasificación básico basado en TF-IDF y una SVM lineal, siguiendo el estilo de ejercicios anteriores:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

model = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LinearSVC())
    ]
)

model.fit(newsgroups_train.data, newsgroups_train.target)
acc = model.score(newsgroups_test.data, newsgroups_test.target)
print("Accuracy on test data", acc)

Ahora vamos a mejorar este Pipeline de clasificación añadiendo variables de LDA. Lo que haremos será exponer al clasificador (LinearSVC) dos grupos de variables explicativas o _features_:

* La vectorización generada por TF-IDF
* Las probabilidades de pertentencia a cada tema detectador por LDA. Nótese que para calcular LDA necesitamos previamente haber hecho la vectorización TF-IDF.

En situaciones como esta en la que queremos proporcionar dos o más conjuntos de variables explicativas al clasificador lo que hacemos es definir un Pipeline que calcule cada conjunto de variables por separado. Empezaremos por el Pipeline de TF-IDF:

In [ ]:
pipeline_tfidf = Pipeline([
    ('vectorizer', TfidfVectorizer())
])

Ahora definimos otro Pipeline que incluya LDA, para lo cual es pre-requisito haber realizado también TF-IDF. Por tanto el Pipeline se compone de estos dos pasos:

In [ ]:
pipeline_lda = Pipeline([
    ('vectorizer', TfidfVectorizer()), 
    ('lda', LatentDirichletAllocation(n_components=20))
])

Una vez tenemos los Pipelines listos podemos combinarlos usando **FeatureUnion**. Un objeto FeatureUnion recibe una lista de parejas, cada parejas siendo el nombre de ese grupo de features y el Pipeline que lo construye. Un FeatureUnion puede a su vez meterse dentro de un Pipeline de modelado, al que luego puede seguir un clasificador. Por tanto para terminar de definir nuestro modelo escribimos lo siguiente:

In [ ]:
from sklearn.pipeline import FeatureUnion

model = Pipeline([
    ('merger', FeatureUnion([
        ('tfidf_pipeline', pipeline_tfidf),
        ('topics_pipeline', pipeline_lda),
    ])),
    ('classifier', LinearSVC())   
])

Ahora entrenamos este modelo y medimos el acierto:

In [ ]:
model.fit(newsgroups_train.data, newsgroups_train.target)
acc = model.score(newsgroups_test.data, newsgroups_test.target)
print("Accuracy on test data", acc)

<table>
<tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">¿Ha mejorado el nivel de acierto tras incluir las variables explicativas basadas en LDA?
</table>

# Otros modelos de detección de topics

Como alternativa a LDA existen otros modelos que también toman como entrada una representación vectorial de un corpus y tratan de inferir los temas subyacentes a ese corpus. En scikit-learn disponemos de los siguientes:

* Latent Semantic Analysis (LSA): implementado en la clase [TruncatedSVD](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD).
* Non-negative Matrix Factorization (NMF): implementado en la clase [NMF](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html#sklearn.decomposition.NMF).

<table>
<tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">Repite el análisis de descomposición de temas anterior con LSA, construyendo un modelo LSA y generando la gráfica de nube de palabras. ¿Observas diferencias con la descomposición en temas obtenida por LDA?
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table>
<tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">Repite el análisis de nuevo con NMF. ¿Qué diferencias observas ahora?
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table>
<tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">Ahora repite el modelo de clasificación supervisada con LDA que utilizamos arriba, pero empleando un modelo LSA en su lugar. ¿Obtienes mejor precisión en test?
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table>
<tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">Repite de nuevo con NMF, ¿qué resultado en test obtienes ahora?
</table>

In [ ]:
####### INSERT YOUR CODE HERE